In [ ]:
import os
import sys
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from dredFISH.Analysis import TissueGraph
from dredFISH.Visualization import viz_cell_layer
# from dredFISH.Utils.__init__plots import *  # import one of the two; but not both
from dredFISH.Utils import powerplots

# import importlib
# importlib.reload(viz_cell_layer)

In [ ]:
# plt.plot(np.arange(4))
# plt.savefig('foo.pdf')

In [ ]:
logging.basicConfig(format='%(asctime)s - %(message)s', 
                datefmt='%m-%d %H:%M:%S', 
                level=logging.INFO,
                )

# Path
basepth = '/bigstore/GeneralStorage/Data/dredFISH/Dataset1-t3'

In [ ]:

# load TMG - with cell layer obs only
TMG = TissueGraph.TissueMultiGraph(basepath=basepth, 
                                   redo=False, # load existing 
                                   quick_load_cell_obs=True,
                                  )

# spatial coordinates
layer = TMG.Layers[0]
XY = layer.XY
x, y = XY[:,0], XY[:,1]

#### a temporary hack
x, y = y, x 
XY = np.vstack([x,y]).T
###

In [ ]:
# define a line to split things into hemi-coronal sections
split_lines = [
    # [(0,0),(1,1)],
    [(550, -6000), (200, 2000)],
    [(200, 2000), (550, -6000)],
    # [(300, 2000), (550, -6000)],
]
cond, isinpoly, XYnew = viz_cell_layer.preview_hemisphere(split_lines, XY=XY)

In [ ]:
%%time
df = viz_cell_layer.generate_default_analysis(split_lines, basepth, XY=XY) 
df

# Viz

In [ ]:
dfpth = os.path.join(basepth, 'default_analysis.csv')
df = pd.read_csv(dfpth, index_col=0)
df

In [ ]:
respth = os.path.join(basepth, 'figures')
if not os.path.isdir(respth):
    os.mkdir(respth)

In [ ]:
%%time
output = os.path.join(respth, 'fig1_basis_space.pdf')
# output = None
powerplots.plot_basis_spatial(df, output=output)


In [ ]:
%%time
dfsub = df[df['hemi']==1]
output = os.path.join(respth, 'fig1-2_basis_space_righthalf.pdf')
# output = None
powerplots.plot_basis_spatial(dfsub, pmode='right_half', xcol='x2', ycol='y2', output=output)

In [ ]:
%%time
output = os.path.join(respth, 'fig2_basis_umap.pdf')
# output = None
powerplots.plot_basis_umap(df, output=output)

In [ ]:
%%time
# # resolutions = np.sort([ for col in typecols])
#     r = float(col[len("type_r"):])

typecols = df.filter(regex='^type_r', axis=1).columns
for i, col in enumerate(typecols):
    hue = col
    output = os.path.join(respth, f'fig3-{i}_{col}.pdf')
    # output = None
    powerplots.plot_type_spatial_umap(df, hue, output=output)

In [ ]:
ktypecols = df.filter(regex='^ktype_L', axis=1).columns
for i, col in enumerate(ktypecols):
    hue = col
    output = os.path.join(respth, f'fig4-{i}_{col}.pdf')
    # output = None
    powerplots.plot_type_spatial_umap(df, hue, output=output)